In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

Using TensorFlow backend.


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1) 

In [3]:
X_train = X_train / 255.0
test = test / 255.0
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [4]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [5]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

In [6]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [7]:
model.compile(optimizer=RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.0),loss="categorical_crossentropy",metrics=["accuracy"])

In [8]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
epochs=30
batch_size=86

In [9]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)
datagen.fit(X_train)

In [10]:
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Epoch 1/30
 - 18s - loss: 0.4118 - acc: 0.8690 - val_loss: 0.0799 - val_acc: 0.9743
Epoch 2/30
 - 11s - loss: 0.1281 - acc: 0.9615 - val_loss: 0.0453 - val_acc: 0.9860
Epoch 3/30
 - 11s - loss: 0.0949 - acc: 0.9725 - val_loss: 0.0344 - val_acc: 0.9890
Epoch 4/30
 - 11s - loss: 0.0787 - acc: 0.9764 - val_loss: 0.0322 - val_acc: 0.9895
Epoch 5/30
 - 11s - loss: 0.0730 - acc: 0.9788 - val_loss: 0.0321 - val_acc: 0.9895
Epoch 6/30
 - 12s - loss: 0.0644 - acc: 0.9813 - val_loss: 0.0330 - val_acc: 0.9898
Epoch 7/30
 - 11s - loss: 0.0585 - acc: 0.9826 - val_loss: 0.0337 - val_acc: 0.9881
Epoch 8/30
 - 11s - loss: 0.0571 - acc: 0.9837 - val_loss: 0.0242 - val_acc: 0.9938
Epoch 9/30
 - 11s - loss: 0.0585 - acc: 0.9831 - val_loss: 0.0182 - val_acc: 0.9952
Epoch 10/30
 - 11s - loss: 0.0631 - acc: 0.9836 - val_loss: 0.0233 - val_acc: 0.9929
Epoch 11/30
 - 11s - loss: 0.0592 - acc: 0.9836 - val_loss: 0.0191 - val_acc: 0.9940
Epoch 12/30
 - 11s - loss: 0.0621 - acc: 0.9842 - val_loss: 0.0234 - val_a

In [11]:
results = model.predict(test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv('submit.csv',index=False)